<a href="https://colab.research.google.com/github/cagBRT/PySpark/blob/master/Distrib_vs_Single.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we compare doing a task in a distributed environment and on a single CPU (single computer)

In [2]:
!git clone https://github.com/cagBRT/PySpark.git

Cloning into 'PySpark'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 420 (delta 56), reused 0 (delta 0), pack-reused 330
Receiving objects: 100% (420/420), 3.37 MiB | 6.92 MiB/s, done.
Resolving deltas: 100% (254/254), done.




---



---



# Setup PySpark<br>

If using DataBricks this section will be different

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b132acd5b101bcbe406bbf642bbc2b35d1e09a0425b37e88050412a0ca1e3702
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
#Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#getOrCreate gets or creates a session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [5]:
import urllib.request
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [6]:
spark = (
    SparkSession.builder
        .appName('learn')
        .config('spark.driver.memory', '8g')
        .master('local[4]',)
        .config('spark.sql.execution.arrow.pyspark.enabled', True)
        .config('spark.sql.execution.arrow.pyspark.fallback.enabled', False)
        .getOrCreate()
)

In [7]:
sc=spark.sparkContext.getOrCreate()



---



---



We will now get all the works of Shakespeare (a very large file) and perform count and sort tasks.

# Get all of Shakespeare’s works

In [8]:
import time

**Let's find out how long it takes a distributed computer to perform the task of counting all the words in Shakespeare's works**.

In [28]:
start_time = time.time()
#Count the number of words in all of Shakespear's works
Words=sc.textFile("/content/PySpark/shakespeare.txt")
WordsCount=Words.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
print("Number of words:",WordsCount.count())

--- 0.04732036590576172 seconds ---
Number of words: 1418390


**Let's find out how long it takes a *single computer* to perform the task of counting all the words in Shakespeare's works**.

In [29]:
start_time = time.time()
f= open("/content/PySpark/shakespeare.txt", "r")
words_shakes= f.read()
f.close()
word_shakes_python=words_shakes.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(word_shakes_python)

--- 0.0781714916229248 seconds ---


1293935

In [30]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.047492265701293945
single compute time:  0.0783390998840332


In [33]:
start_time = time.time()
#Count the number of distinct words
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
print("Number of distinct words:",DistinctWordsCount.count())

--- 0.021169424057006836 seconds ---
Number of distinct words: 67506


In [34]:
start_time = time.time()
words_unique=set(word_shakes_python)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

--- 0.06757402420043945 seconds ---


86196

In [35]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.02133345603942871
single compute time:  0.06775259971618652


In [36]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
SortedWordsCount.top(20)

--- 0.9317646026611328 seconds ---


[(517065, ''),
 (23242, 'the'),
 (19540, 'I'),
 (18297, 'and'),
 (15623, 'to'),
 (15544, 'of'),
 (12532, 'a'),
 (10824, 'my'),
 (9576, 'in'),
 (9081, 'you'),
 (7851, 'is'),
 (7531, 'that'),
 (7068, 'And'),
 (6948, 'not'),
 (6722, 'with'),
 (6218, 'his'),
 (6009, 'your'),
 (6002, 'be'),
 (5616, 'for'),
 (5236, 'have')]

In [37]:
#start_time = time.time()
#counted_words=[]
#words_unique=list(words_unique)
#for i in range(len(words_unique)):
#  tuples= (word_shakes_python.count(words_unique[i]), words_unique[i])
#  counted_words.append(tuples)
#print("--- %s seconds ---" % (time.time() - start_time))
#sc_time= time.time() - start_time

In [38]:
#print("distributed time:    ",dc_time )
#print("single compute time: ", sc_time)

distributed time:     1.38<br>
single compute time:  2158



---

# Gettyburg Address

---



**Count all the words using a distributed environment**

In [39]:
start_time = time.time()
Lincoln = sc.textFile("/content/PySpark/GettysBurg.txt")
LincolnCount=Lincoln.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
dist_lincoln_count=LincolnCount.count()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
print("Number of words:",dist_lincoln_count)

--- 0.3471508026123047 seconds ---
Number of words: 266


**Count all the words using a single CPU**

In [40]:
start_time = time.time()
f= open("/content/PySpark/GettysBurg.txt", "r")
single_lincoln_count= f.read()
f.close()
single_lincoln_count=single_lincoln_count.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(single_lincoln_count)

--- 0.0016939640045166016 seconds ---


263

In [41]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.3473639488220215
single compute time:  0.002080678939819336


**Count unique words using a distributed environment**

In [21]:
#Count the number of distinct words
start_time = time.time()
DistinctWordsCount=LincolnCount.reduceByKey(lambda a,b: a+b)
DistinctWordsCount.count()
dc_time=time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))
DistinctWordsCount.count()

--- 0.8372633457183838 seconds ---


158

**Count the unique words using a single computer**

In [22]:
start_time = time.time()
words_unique=set(single_lincoln_count)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

--- 0.00017571449279785156 seconds ---


155

In [23]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.8371624946594238
single compute time:  0.0002894401550292969


**Count the word frequency using a distributed environment**

In [24]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
#SortedWordsCount.top(20)

--- 0.8663744926452637 seconds ---


**Count the word frequency using a single computer**

In [45]:
start_time = time.time()
counted_words=[]
words_unique=list(words_unique)
for i in range(len(words_unique)):
  tuples= (words_unique.count(words_unique[i]), words_unique[i])
  counted_words.append(tuples)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time= time.time() - start_time


KeyboardInterrupt: 

In [26]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.8669850826263428
single compute time:  3.084928035736084
